# VectorDB_Embedding１：ダイレクトにIndexから文書を選びベクトル化

In [1]:
!pip uninstall -y openai
!pip install openai==0.28.0

!pip install tiktoken
!pip install python-dotenv
!pip install pinecone==6.0.1

Found existing installation: openai 1.68.2
Uninstalling openai-1.68.2:
  Successfully uninstalled openai-1.68.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 16.4 MB/s eta 0:00:00


In [ ]:
import requests
import re
import os
import csv
import time  # ★ sleep用
import pandas as pd
from urllib.parse import urlsplit
from bs4 import BeautifulSoup

import openai
import tiktoken
from dotenv import load_dotenv
from pinecone import Pinecone
from pinecone import NotFoundException  # ← 例外ハンドリング用

############################
# 1) APIキー読み込み
############################
def load_api_keys(txt_filepath="api_keys.txt"):
    if not os.path.exists(txt_filepath):
        raise FileNotFoundError(f"APIキーのファイルが見つかりません: {txt_filepath}")

    # BOM 付き UTF-8 でも読めるように encoding="utf-8"
    with open(txt_filepath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k.strip()] = v.strip()

load_api_keys("api_keys.txt")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")
OPENAI_API_KEY   = os.getenv("OPENAI_API_KEY", "")
openai.api_key   = OPENAI_API_KEY

INDEX_NAME = "concur-index"
NAMESPACE  = "demo-html"

############################
# 2) Pinecone 初期化
############################
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

# ▼▼ ここで全部消していた処理をコメントアウト ▼▼
# try:
#     index.delete(deleteAll=True, namespace=NAMESPACE)
#     print(f"[INFO] Deleted all vectors in namespace '{NAMESPACE}'.")
# except NotFoundException:
#     print(f"[WARN] '{NAMESPACE}' namespace not found. (No problem, skipping.)")

stats = index.describe_index_stats()
ns_info = stats.get("namespaces", {})
if NAMESPACE in ns_info:
    print(f"[INFO] namespace '{NAMESPACE}' 既に存在: vector_count={ns_info[NAMESPACE]['vector_count']}")
else:
    print(f"[INFO] namespace '{NAMESPACE}' はまだ存在しません。")


############################
# 3) indexページをパース (HTMLスクレイプ)
############################
def parse_index_page(index_url):
    resp = requests.get(index_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    result_list = []
    table = soup.find("table")
    if not table:
        print("[WARN] テーブルが見つかりません:", index_url)
        return result_list

    for row in table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) < 4:
            continue
        guide_en = cols[0].get_text(strip=True)
        guide_jp = cols[1].get_text(strip=True)
        base_url = cols[3].get_text(strip=True)
        result_list.append({
            "GuideNameEn": guide_en,
            "GuideNameJp": guide_jp,
            "BaseURL": base_url
        })
    return result_list


############################
# 4) mapping_DB.csv ロード
############################
def load_mapping_db(csv_path="mapping_DB.csv"):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"[ERROR] mapping_DB.csv が見つかりません: {csv_path}")

    rows = []
    with open(csv_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f, delimiter=",")
        for row in reader:
            rows.append(row)
    return rows


############################
# 5) HTMLソース取得
############################
def fetch_html(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text

############################
# 6) アンカーoffset検索
############################
def find_anchor_offset(html_src, anchor_id):
    pattern = re.compile(
        rf'<a[^>]+(?:id|name)\s*=\s*["\']{re.escape(anchor_id)}["\']',
        re.IGNORECASE
    )
    m = pattern.search(html_src)
    if m:
        return m.start()
    return -1

############################
# 7) chunking & embedding
############################
def get_embedding(text: str, model="text-embedding-ada-002"):
    resp = openai.Embedding.create(model=model, input=text)
    return resp["data"][0]["embedding"]

def chunk_text(text: str, chunk_size=2000):
    chunks = []
    start = 0
    length = len(text)
    while start < length:
        end = min(start + chunk_size, length)
        chunks.append(text[start:end])
        start = end
    return chunks


############################
# メインフロー
############################
def main():
    # 1) indexページをスクレイプしてガイド一覧を取得
    index_url = "https://la-concur-helper.github.io/concur-docs/index.htm"
    docs = parse_index_page(index_url)
    if not docs:
        print("[ERROR] indexページからガイド情報が取得できませんでした。")
        return

    print("\n=== ガイド一覧 ===")
    for i, d in enumerate(docs, start=1):
        print(f"{i}. {d['GuideNameEn']} / {d['GuideNameJp']} => {d['BaseURL']}")

    sel = input("\nどのガイドを処理しますか？(番号): ").strip()
    if not sel.isdigit():
        print("キャンセル")
        return
    idx = int(sel)
    if idx < 1 or idx > len(docs):
        print("[ERROR] 選択が範囲外。終了。")
        return

    chosen = docs[idx-1]
    base_url = chosen["BaseURL"]
    guide_en = chosen["GuideNameEn"]
    guide_jp = chosen["GuideNameJp"]

    # e.g. "Exp_SG_Account_Codes-jp.html" -> "Exp_SG_Account_Codes-jp.docx"
    fname = os.path.basename(urlsplit(base_url).path)
    doc_name = re.sub(r"\.html?$", ".docx", fname, flags=re.IGNORECASE)

    print(f"\n選択されたガイド => doc_name={doc_name}, base_url={base_url}")

    # 2) mapping_DB.csv をロード
    mapping_rows = load_mapping_db("mapping_DB.csv")
    if not mapping_rows:
        print("[WARN] mapping_DB.csv から行が読み込めませんでした。")
        return

    # doc_name 一致行を抽出
    doc_rows = [r for r in mapping_rows if r.get("DocName","") == doc_name]
    if not doc_rows:
        print(f"[WARN] DocName='{doc_name}' の行がmapping_DB.csvにありません。終了。")
        return

    # 3) HTMLを取得
    html_src = fetch_html(base_url)
    print(f"[INFO] HTMLソース取得: 長さ={len(html_src)}")

    # 4) doc_rows を PageNumber + AnchorID でソート
    def sort_key(row):
        p = 999999
        if row["PageNumber"].isdigit():
            p = int(row["PageNumber"])
        return (p, row["AnchorID"])
    sorted_rows = sorted(doc_rows, key=sort_key)

    # 5) offset計算
    offsets = []
    for row in sorted_rows:
        anchor_id = row["AnchorID"]
        off = find_anchor_offset(html_src, anchor_id)
        offsets.append({ "row": row, "offset": off })

    ############################
    # ▼▼ バッチアップサート ▼▼
    ############################

    BATCH_SIZE = 100
    vectors_buffer = []
    doc_id_counter = 0

    def flush_upsert_buffer():
        nonlocal vectors_buffer
        if not vectors_buffer:
            return
        print(f"[BATCH UPSERT] {len(vectors_buffer)} vectors ...")
        resp = index.upsert(vectors=vectors_buffer, namespace=NAMESPACE)
        print("[BATCH RESP]:", resp)
        vectors_buffer = []
        time.sleep(1)

    # offset[i]~offset[i+1] で区間抽出 => chunk化 => embedding => upsert
    for i in range(len(offsets)):
        cur = offsets[i]
        row  = cur["row"]
        off_i = cur["offset"]
        if off_i < 0:
            off_i = 0

        if i < len(offsets)-1:
            off_j = offsets[i+1]["offset"]
            if off_j < 0:
                off_j = len(html_src)
        else:
            off_j = len(html_src)

        if off_j < off_i:
            off_j = off_i

        raw_segment = html_src[off_i:off_j]
        text_str = re.sub(r"<[^>]+>", "", raw_segment)
        text_str = re.sub(r"\s+", " ", text_str).strip()
        if not text_str:
            continue

        doc_name_val = row["DocName"]
        guide_jp_val = row["GuideNameJp"]
        sec1         = row["SectionTitle1"]
        sec2         = row["SectionTitle2"]
        anchor_val   = row["AnchorID"]
        link_val     = row["FullLink"]

        # セクションタイトルと本文の間に改行
        combined_text = f"{sec1}\n{sec2}\n\n{text_str}"

        chunk_list = chunk_text(combined_text, chunk_size=2000)

        for chunk_str_data in chunk_list:
            doc_id_counter += 1
            chunk_id = f"{doc_name_val}_chunk{doc_id_counter}"

            emb = get_embedding(chunk_str_data)
            meta = {
                "DocName": doc_name_val,
                "GuideNameJp": guide_jp_val,
                "SectionTitle1": sec1,
                "SectionTitle2": sec2,
                "AnchorID": anchor_val,
                "FullLink": link_val,
                "chunk_text": chunk_str_data
            }

            vectors_buffer.append({
                "id": chunk_id,
                "values": emb,
                "metadata": meta
            })

            if len(vectors_buffer) >= BATCH_SIZE:
                flush_upsert_buffer()

    # バッファに残ったデータを最後にアップサート
    flush_upsert_buffer()
    print("[INFO] 全アップサート完了")

    # テスト検索
    test_query = "ConcurのAPIでレシート登録したい"
    print(f"\n[INFO] テスト検索: {test_query}")
    q_emb = get_embedding(test_query)
    sr = index.query(
        vector=q_emb,
        top_k=3,
        include_metadata=True,
        namespace=NAMESPACE
    )
    for match in sr.matches:
        md = match.metadata
        print(f"- ID={match.id}, Score={match.score}")
        print("  SectionTitle2:", md.get("SectionTitle2"))
        print("  FullLink:", md.get("FullLink"))
        print("  chunk_text:", md.get("chunk_text","")[:80], "...\n")


if __name__ == "__main__":
    main()

[INFO] namespace 'demo-html' 既に存在: vector_count=2251

=== ガイド一覧 ===
1. Guide Name / ガイド名 => URL
2. Account Codes / 勘定科目コード(2017年01月10日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Account_Codes-jp.html
3. Allocations / 配賦(2020年7月1日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Allocations-jp.html
4. Attendees / 同席者(2023年6月28日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Attendees-jp.html
5. Audit Rules / 監査ルール(2024年8月17日版)別冊:日付の演算子について => https://la-concur-helper.github.io/concur-docs/Exp_SG_Audit_Rules-jp.html
6. Audit Rules (Validation Rules) / 監査ルール（検証ルール）(2022年8月5日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Audit_Rules_Validation_Rules-jp.html
7. Budget (Shared) / 予算（製品共通）(2021年2月20日版)別冊:予算管理についてのよくある質問(2019年5月29日版) => https://la-concur-helper.github.io/concur-docs/Shr_SG_Budget-jp.html
8. Car Configuration / 車両設定(2021年1月27日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Car_Config-jp.html
9. Cash Advance / 仮払申請(2

## VectorDB_Embedding2：手元のWORD文書からベクトル化

In [ ]:
!pip uninstall -y openai
!pip install openai==0.28.0

!pip install tiktoken
!pip install python-dotenv
!pip install pinecone==6.0.1
!pip install python-docx

import requests
import re
import os
import csv
import docx
import pandas as pd
import time  # ← バッチアップサート時に少し待機するため
from urllib.parse import urlsplit
from bs4 import BeautifulSoup

import openai
import tiktoken
from dotenv import load_dotenv
from pinecone import Pinecone

############################
# 1) APIキー読み込み
############################
def load_api_keys(txt_filepath="api_keys.txt"):
    if not os.path.exists(txt_filepath):
        raise FileNotFoundError(f"APIキーのファイルが見つかりません: {txt_filepath}")

    with open(txt_filepath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k.strip()] = v.strip()

load_api_keys("api_keys.txt")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")
OPENAI_API_KEY   = os.getenv("OPENAI_API_KEY", "")
openai.api_key   = OPENAI_API_KEY

INDEX_NAME = "concur-index"
NAMESPACE  = "demo-html"

############################
# 2) Pinecone 初期化
############################
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

stats = index.describe_index_stats()
ns_info = stats.get("namespaces", {})
if NAMESPACE in ns_info:
    print(f"[INFO] namespace '{NAMESPACE}' 既に存在: vector_count={ns_info[NAMESPACE]['vector_count']}")
else:
    print(f"[INFO] namespace '{NAMESPACE}' はまだ存在しません。")


############################
# 3) indexページ (HTML) をパース
############################
def parse_index_page(index_url):
    resp = requests.get(index_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    result_list = []
    table = soup.find("table")
    if not table:
        print("[WARN] テーブルが見つかりません:", index_url)
        return result_list

    for row in table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) < 4:
            continue
        guide_en = cols[0].get_text(strip=True)
        guide_jp = cols[1].get_text(strip=True)
        base_url = cols[3].get_text(strip=True)
        result_list.append({
            "GuideNameEn": guide_en,
            "GuideNameJp": guide_jp,
            "BaseURL": base_url
        })
    return result_list

############################
# 4) mapping_DB.csv ロード
############################
def load_mapping_db(csv_path="mapping_DB.csv"):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"[ERROR] mapping_DB.csv が見つかりません: {csv_path}")

    rows = []
    with open(csv_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f, delimiter=",")
        for row in reader:
            rows.append(row)
    return rows

############################
# 5) HTMLソース取得
############################
def fetch_html(url):
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    return resp.text

############################
# 6) アンカーoffset検索
############################
def find_anchor_offset(html_src, anchor_id):
    pattern = re.compile(
        rf'<a[^>]+(?:id|name)\s*=\s*["\']{re.escape(anchor_id)}["\']',
        re.IGNORECASE
    )
    m = pattern.search(html_src)
    if m:
        return m.start()
    return -1

############################
# 7) chunking & embedding
############################
def get_embedding(text: str, model="text-embedding-ada-002"):
    resp = openai.Embedding.create(model=model, input=text)
    return resp["data"][0]["embedding"]

def chunk_text(text: str, chunk_size=2000):
    chunks = []
    start = 0
    length = len(text)
    while start < length:
        end = min(start + chunk_size, length)
        chunks.append(text[start:end])
        start = end
    return chunks

############################
# 8) Word文書(ローカル) からテキスト抽出
############################
def parse_local_word(docx_path):
    if not os.path.exists(docx_path):
        raise FileNotFoundError(f"[ERROR] Word文書が見つかりません: {docx_path}")

    doc = docx.Document(docx_path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    full_text = "\n".join(paragraphs)
    return full_text


def main():
    # 1) indexページからdocリストを取得
    index_url = "https://la-concur-helper.github.io/concur-docs/index.htm"
    docs = parse_index_page(index_url)
    if not docs:
        print("[ERROR] indexページが見つかりません。")
        return

    print("\n=== ガイド一覧 ===")
    for i, d in enumerate(docs, start=1):
        print(f"{i}. {d['GuideNameEn']} / {d['GuideNameJp']} => {d['BaseURL']}")

    sel = input("\nどのガイドを処理しますか？(番号): ").strip()
    if not sel.isdigit():
        print("[ERROR] キャンセル or 無効入力")
        return
    idx = int(sel)
    if idx < 1 or idx > len(docs):
        print("[ERROR] 範囲外")
        return

    chosen = docs[idx-1]
    base_url = chosen["BaseURL"]
    guide_en = chosen["GuideNameEn"]
    guide_jp = chosen["GuideNameJp"]

    # 例: "Exp_SG_Account_Codes-jp.html" => "Exp_SG_Account_Codes-jp.docx"
    fname = os.path.basename(urlsplit(base_url).path)
    doc_name = re.sub(r"\.html?$", ".docx", fname, flags=re.IGNORECASE)

    print(f"\n選択ガイド => doc_name={doc_name}, base_url={base_url}")

    # 2) mapping_DB.csv ロード
    mapping_rows = load_mapping_db("mapping_DB.csv")
    if not mapping_rows:
        print("[ERROR] mapping_DB.csv 読み込み失敗")
        return

    # doc_name に一致する行
    doc_rows = [r for r in mapping_rows if r.get("DocName", "") == doc_name]

    ############################
    # ▼▼ バッチアップサート ▼▼
    ############################

    # Pineconeアップサート用の設定
    BATCH_SIZE = 100
    vectors_buffer = []
    doc_id_counter = 0

    def flush_upsert_buffer():
        """バッファの内容をまとめてUpsertし、バッファをクリア"""
        nonlocal vectors_buffer
        if not vectors_buffer:
            return
        print(f"[BATCH UPSERT] {len(vectors_buffer)} vectors ...")
        resp = index.upsert(vectors=vectors_buffer, namespace=NAMESPACE)
        print("[BATCH RESP]:", resp)
        # バッファをクリア
        vectors_buffer = []
        # 連打を避けるため少し待機
        time.sleep(1)

    # ================================
    # (A) HTMLダウンロードの試行
    # ================================
    can_download = True
    html_src = ""
    try:
        print(f"[INFO] ダウンロード試行: {base_url}")
        html_src = fetch_html(base_url)
    except Exception as e:
        print(f"[WARN] ダウンロード失敗: {e}")
        can_download = False

    if can_download and doc_rows:
        # --- HTMLが成功ダウンロード → mapping_DBに従い、アンカー単位で分割 ---
        print("[INFO] Webダウンロード成功 → mapping_DBに従い、アンカー単位で分割します。")

        # PageNumber + AnchorID でソート
        def sort_key(r):
            p = 999999
            if r["PageNumber"].isdigit():
                p = int(r["PageNumber"])
            return (p, r["AnchorID"])
        sorted_rows = sorted(doc_rows, key=sort_key)

        offsets = []
        for row in sorted_rows:
            anchor_id = row["AnchorID"]
            off = find_anchor_offset(html_src, anchor_id)
            offsets.append({"row": row, "offset": off})

        for i in range(len(offsets)):
            cur = offsets[i]
            r   = cur["row"]
            off_i = cur["offset"]
            if off_i < 0:
                off_i = 0

            if i < len(offsets)-1:
                off_j = offsets[i+1]["offset"]
                if off_j < 0:
                    off_j = len(html_src)
            else:
                off_j = len(html_src)
            if off_j < off_i:
                off_j = off_i

            raw_segment = html_src[off_i:off_j]
            text_str = re.sub(r"<[^>]+>", "", raw_segment)
            text_str = re.sub(r"\s+", " ", text_str).strip()
            if not text_str:
                continue

            chunk_list = chunk_text(text_str, chunk_size=2000)

            doc_name_val = r["DocName"]
            guide_jp_val = r["GuideNameJp"]
            sec1         = r["SectionTitle1"]
            sec2         = r["SectionTitle2"]
            anchor_val   = r["AnchorID"]
            link_val     = r["FullLink"]

            for chunk_str_data in chunk_list:
                doc_id_counter += 1
                chunk_id = f"{doc_name_val}_chunk{doc_id_counter}"

                emb = get_embedding(chunk_str_data)
                meta = {
                    "DocName": doc_name_val,
                    "GuideNameJp": guide_jp_val,
                    "SectionTitle1": sec1,
                    "SectionTitle2": sec2,
                    "AnchorID": anchor_val,
                    "FullLink": link_val,
                    "chunk_text": chunk_str_data
                }

                vectors_buffer.append({
                    "id": chunk_id,
                    "values": emb,
                    "metadata": meta
                })

                # バッファが一定サイズに達したらUpsert
                if len(vectors_buffer) >= BATCH_SIZE:
                    flush_upsert_buffer()

        # ループ完了後、残りがあればフラッシュ
        flush_upsert_buffer()
        print("[INFO] HTML文書のアップサート完了.")

    else:
        # ================================
        # (B) Word文書がローカルにあるパターン
        # ================================
        print("\n[INFO] Word文書のローカルアップロード対応に切り替えます。")
        local_path = input("Word文書 (.docx) のローカルファイルパスを入力してください: ").strip()
        if not os.path.exists(local_path):
            print(f"[ERROR] 指定ファイルが見つかりません: {local_path}")
            return

        # parse_local_word でテキスト全抽出
        full_text = parse_local_word(local_path)
        print(f"[INFO] Word文書の段落合計文字数={len(full_text)}")

        # chunk化 & Embedding
        chunked_data = chunk_text(full_text, chunk_size=2000)

        doc_name_val = doc_name
        guide_jp_val = guide_jp

        for chunk_str_data in chunked_data:
            doc_id_counter += 1
            chunk_id = f"{doc_name_val}_chunk{doc_id_counter}"

            emb = get_embedding(chunk_str_data)
            meta = {
                "DocName": doc_name_val,
                "GuideNameJp": guide_jp_val,
                "FullLink": "(Local file, no link)",
                "chunk_text": chunk_str_data
            }
            vectors_buffer.append({
                "id": chunk_id,
                "values": emb,
                "metadata": meta
            })

            # バッファが一定サイズに達したらUpsert
            if len(vectors_buffer) >= BATCH_SIZE:
                flush_upsert_buffer()

        # ループ完了後、残りがあればフラッシュ
        flush_upsert_buffer()
        print("[INFO] ローカルWord文書のアップサート完了.")

    # テスト検索
    test_query = "ConcurのAPIでレシート登録したい"
    print(f"\n[INFO] テスト検索: {test_query}")
    q_emb = get_embedding(test_query)
    sr = index.query(
        vector=q_emb,
        top_k=3,
        include_metadata=True,
        namespace=NAMESPACE
    )
    for match in sr.matches:
        md = match.metadata
        print(f"- ID={match.id}, Score={match.score}")
        print("  DocName:", md.get("DocName"))
        print("  chunk_text:", md.get("chunk_text","")[:80], "...")
        print("")

if __name__ == "__main__":
    main()


Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Successfully uninstalled openai-0.28.0
  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)


ERROR: Operation cancelled by user
^C


FileNotFoundError: APIキーのファイルが見つかりません: api_keys.txt